# 8. Statistics & supervised machine learning

Explanation vs. prediction

TO DECIDE
- ...

### Text book

- Computer Age Statistical Inference

### Notes

- Check https://doi.org/10.1145/1150402.1150412


## 9.1. Explanation (statistical modeling)

Example: TweetsCOV19
- Explain number of tweet's retweets by message and user features

https://jakevdp.github.io/PythonDataScienceHandbook/05.06-linear-regression.html

In [ ]:
# model fit r^2
# variable transformation
# hypothesis and significance testing
# generalized linear model: linear and logistic regression

## 9.2. Prediction (supervised machine learning)

- Out-of-sample validation
- Train/test splitting
- Cross validation
- Data leakage
- Feature selection
- Over- and underfitting

- https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html


In [1]:
# linear and logistic regression, compare results

BOXES: REFER TO SUPPORT VECTOR MACHINES AND NAIVE BAYES CLASSIFICATION

https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html
https://jakevdp.github.io/PythonDataScienceHandbook/05.05-naive-bayes.html

## 9.3. Decision trees and random forests

https://jakevdp.github.io/PythonDataScienceHandbook/05.08-random-forests.html

## 9.4. Towards deep learning

Check https://doi.org/10.1073/pnas.1218772110

- Convey an understanding of neural networks and deep learning
- Start by extendin features by hashtag/named entity/mention representations
- Use neural network off the sklearn shelf

## 9.5. Model selection

- By interpretability
- By predictive accuracy